# Analyze Results

In [ ]:
import os.path
import pandas as pd

from dbispipeline.analytics import get_results_as_dataframe
from hit_prediction_code.analytics import add_approach_to_df
from hit_prediction_code.analytics import add_cv_epoch_evaluator_outcome_to_df

results = get_results_as_dataframe(project_name='hit-prediction-ismir2020', table_name='hit_prediction', filter_git_dirty=False)
results = results[results['date'] > '2021-02-02']

add_approach_to_df(results)
add_cv_epoch_evaluator_outcome_to_df(results)

results.sort_values(by='approach', inplace=True)

In [ ]:
import numpy as np

from hit_prediction_code.analytics import plot_epochs_confution_matrix

for _, result in results.iterrows():
    mean_result = result['mean']
    
    cm_results = mean_result.loc['confusion_matrix']
    
    target_counts = np.sum(cm_results[0], axis=1) + 1
    divisor = np.broadcast_to(target_counts + 1, (len(target_counts), (len(target_counts)))).transpose()

    normalized_cm = []

    for cm in cm_results:
        cm = np.array(cm)
        normalized_cm.append(cm / divisor)
    
    plot_shape = (2, 3)
    
    if len(normalized_cm) >= plot_shape[0] * plot_shape[1]:
        plot_epochs_confution_matrix(result['approach'], normalized_cm, mean_result.columns, plot_shape=plot_shape, figsize=(24, 14))